In [67]:
# Required Modules
import pandas as pd
import sqlite3
from sqlite3 import Error
import os

# Identify database path
db_path = os.path.abspath('./sqlite.db')

# Connect to SQLite Database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

sqlite3.sqlite_version

'3.32.3'

In [68]:
# FL Zip Codes from Census data for reference
fl_census_path = os.path.abspath('Resources/ACSST5Y2019.S1901_data_with_overlays_2021-05-24T165952.csv')
fl_census_df = pd.read_csv(fl_census_path, header=1)
fl_codes = []

for i,j in fl_census_df.iterrows():
    fl_codes.append(j['Geographic Area Name'][6:11])

fl_zipcodes = pd.DataFrame(fl_codes)
fl_zipcodes = fl_zipcodes.rename(columns={0:'zipcode'})

# Drop table if it exist
cursor.execute("DROP TABLE IF EXISTS zipcode")

# Create table
fl_zipcodes.to_sql('zipcode', conn, if_exists='replace', index=False)
fl_zipcodes['zipcode'].nunique()

984

In [69]:
# Sales data
sales_path = os.path.abspath('Resources/Sales_Area_Data/Sales/final_sales_data.csv')
sales_data_df = pd.read_csv(sales_path, header=0)
sales_data_df = sales_data_df[['zipcode','year','month','total_sales','avg_sale_price','med_sale_price','mortgage_rate']]

# Drop table if it exist
cursor.execute("DROP TABLE IF EXISTS sales")

# Create table
sales_data_df.to_sql('sales', conn, if_exists='replace', index=False)
sales_data_df['zipcode'].nunique()

962

In [70]:
# Sales data
area_path = os.path.abspath('Resources/Sales_Area_Data/Area/final_area_data.csv')
area_data_df = pd.read_csv(area_path, header=0)
area_data_df = area_data_df[['zipcode','property_tax','owner_occupied','renter_occupied','total_vacant','total_dwellings','studio_rent','one_bed_rent','two_bed_rent','three_bed_rent','four_bed_rent','fte_employed','unemployed','average_income','projected_income','expense_index','average_commute','crime_index']]
area_data_df.head()

# Drop table if it exist
cursor.execute("DROP TABLE IF EXISTS area")

# Create table
area_data_df.to_sql('area', conn, if_exists='replace', index=False)
area_data_df['zipcode'].nunique()

949

In [71]:
tables_ls = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
tables_ls

,name
0,mobility
1,household_income
2,mobility_slim
3,grades
4,fha_loans
5,zipcode
6,sales
7,area


In [72]:
df = pd.read_sql('''SELECT      zip.zipcode AS Zip_Code, 
                                date((cast(y.year as text)||"-"||substr("0"||cast(m.month as text),-2,2)||"-01")) AS Date,
                                y.year AS Year, 
                                m.month AS Month,
                                ifnull(s.med_sale_price,0) AS Sale_Price,
                                ifnull(s.mortgage_rate,0) AS Interest_Rate,
                                ifnull(a.property_tax,0) AS Property_Tax,
                                ifnull(a.two_bed_rent,0) AS Rent_Price,
                                ifnull(inc.B19013001,0) AS Household_Income,
                                ifnull(a.two_bed_rent/(inc.B19013001/12),0) AS Rent_Affordability,
                                ifnull(s.total_sales,0) AS Total_Sales,
                                ifnull(fha.fha,0) AS FHA_Count,
                                ifnull(mob.mobility_rate,0) AS Mobility_Rate, 
                                ifnull(g.percent_total_points,0) AS School_Rating,
                                ifnull(g.letter_grade,"Unknown") AS School_Grade,
                                ifnull(a.owner_occupied,0) AS Owner_Occupied,
                                ifnull(a.renter_occupied,0) AS Renter_Occupied,
                                ifnull(a.total_vacant,0) AS Total_Vacant,
                                ifnull(a.total_dwellings,0) AS Total_Dwellings,
                                ifnull(a.fte_employed,0) AS FTE_Employed,
                                ifnull(a.unemployed,0) AS Unemployed,
                                ifnull(a.expense_index,0) AS Expense_Index,
                                ifnull(a.average_commute,0) AS Average_Commute,
                                ifnull(a.crime_index,0) AS Crime_Index
                        FROM zipcode AS zip
                        CROSS JOIN (SELECT 2019 AS year UNION SELECT 2020 AS year UNION SELECT 2021 AS year) AS y
                        CROSS JOIN (SELECT 1 AS month UNION SELECT 2 AS month UNION SELECT 3 AS month
                                UNION SELECT 4 AS month UNION SELECT 5 AS month UNION SELECT 6 AS month
                                UNION SELECT 7 AS month UNION SELECT 8 AS month UNION SELECT 9 AS month
                                UNION SELECT 10 AS month UNION SELECT 11 AS month UNION SELECT 12 AS month) AS m
                        LEFT JOIN fha_loans AS fha ON zip.zipcode = fha.zipcode AND y.year = fha.year AND m.month = fha.month
                        LEFT JOIN mobility_slim AS mob ON zip.zipcode = mob.name
                        LEFT JOIN household_income AS inc ON zip.zipcode = inc.name
                        LEFT JOIN grades AS g ON zip.zipcode = g.zip
                        LEFT JOIN sales AS s ON zip.zipcode = s.zipcode AND y.year = s.year AND m.month = s.month
                        LEFT JOIN area AS a ON zip.zipcode = a.zipcode
                        WHERE s.med_sale_price IS NOT NULL
                        AND s.mortgage_rate IS NOT NULL
                        AND IFNULL(inc.B19013001,0) > 0
                        AND a.two_bed_rent IS NOT NULL
                        AND y.year IS NOT NULL
                        AND m.month IS NOT NULL
                        AND NOT (y.year = 2021 AND m.month = 4)
                ''', conn)
df.head()

,Zip_Code,Date,Year,Month,Sale_Price,Interest_Rate,Property_Tax,Rent_Price,Household_Income,Rent_Affordability,...,School_Grade,Owner_Occupied,Renter_Occupied,Total_Vacant,Total_Dwellings,FTE_Employed,Unemployed,Expense_Index,Average_Commute,Crime_Index
0,32003,2019-01-01,2019,1,244950.0,4.4640,3778,1113,94154.0,0.141853,...,Unknown,8834,2186,704,11724,10671,1069,104,35,28
1,32003,2019-02-01,2019,2,270000.0,4.3700,3778,1113,94154.0,0.141853,...,Unknown,8834,2186,704,11724,10671,1069,104,35,28
2,32003,2019-03-01,2019,3,275000.0,4.2650,3778,1113,94154.0,0.141853,...,Unknown,8834,2186,704,11724,10671,1069,104,35,28
3,32003,2019-04-01,2019,4,264500.0,4.1425,3778,1113,94154.0,0.141853,...,Unknown,8834,2186,704,11724,10671,1069,104,35,28
4,32003,2019-05-01,2019,5,281000.0,4.0720,3778,1113,94154.0,0.141853,...,Unknown,8834,2186,704,11724,10671,1069,104,35,28


In [73]:
# Calculate Home Affordability

# Loan Payment calc source: https://medium.com/personal-finance-analytics/mortgage-calculator-python-code-94d976d25a27
# df = df.append({'Zip_Code':32792,'Year':2011,'Month':10,'Sale_Price':178750,'Interest_Rate':2.875}, ignore_index=True) # PMT should = 593
df['Loan_Amount'] = df['Sale_Price']*.80 # Assuming 20% down payment
df['Loan_Term'] = float(30*12) # Assuming 30 year loan
df['Loan_R'] = 1+(df['Interest_Rate']/100/12)
df['Loan_Payment'] = df['Loan_Amount']*(df['Loan_R']**df['Loan_Term'])*(1-df['Loan_R'])/(1-df['Loan_R']**df['Loan_Term'])
# df['Home_Affordability'] = (df['Loan_Payment']+(df['Property_Tax']/12))/df['Household_Income']
df['Home_Affordability'] = (df['Loan_Payment']*12)/df['Household_Income']

df.tail()

,Zip_Code,Date,Year,Month,Sale_Price,Interest_Rate,Property_Tax,Rent_Price,Household_Income,Rent_Affordability,...,FTE_Employed,Unemployed,Expense_Index,Average_Commute,Crime_Index,Loan_Amount,Loan_Term,Loan_R,Loan_Payment,Home_Affordability
24377,34997,2021-01-01,2021,1,305000.0,2.7350,3067,1211,58789.0,0.247189,...,15421,1267,91,24,113,244000.0,360.0,1.002279,994.170908,0.202930
24378,34997,2021-02-01,2021,2,300000.0,2.8100,3067,1211,58789.0,0.247189,...,15421,1267,91,24,113,240000.0,360.0,1.002342,987.423019,0.201553
24379,34997,2021-02-01,2021,2,300000.0,2.8100,3067,1211,58789.0,0.247189,...,15421,1267,91,24,113,240000.0,360.0,1.002342,987.423019,0.201553
24380,34997,2021-03-01,2021,3,599000.0,3.0825,3067,1211,58789.0,0.247189,...,15421,1267,91,24,113,479200.0,360.0,1.002569,2041.710565,0.416754
24381,34997,2021-03-01,2021,3,599000.0,3.0825,3067,1211,58789.0,0.247189,...,15421,1267,91,24,113,479200.0,360.0,1.002569,2041.710565,0.416754


In [74]:
# Push to CSV
csv_path = os.path.abspath('Resources/Final_Data.csv')
df.to_csv(csv_path, index=False)

In [75]:
# Close connection to SQLite Database
if conn:
    conn.close()

In [76]:
# Check unique counts
df.nunique()

Zip_Code                909
Date                     27
Year                      3
Month                    12
Sale_Price             4736
Interest_Rate            27
Property_Tax            828
Rent_Price               40
Household_Income        903
Rent_Affordability      909
Total_Sales             216
FHA_Count                98
Mobility_Rate           900
School_Rating            52
School_Grade              5
Owner_Occupied          881
Renter_Occupied         846
Total_Vacant            803
Total_Dwellings         896
FTE_Employed            882
Unemployed              748
Expense_Index            92
Average_Commute          32
Crime_Index             439
Loan_Amount            4736
Loan_Term                 1
Loan_R                   27
Loan_Payment          15335
Home_Affordability    23507
dtype: int64

In [77]:
# Check for nulls
df.isnull().sum()

Zip_Code              0
Date                  0
Year                  0
Month                 0
Sale_Price            0
Interest_Rate         0
Property_Tax          0
Rent_Price            0
Household_Income      0
Rent_Affordability    0
Total_Sales           0
FHA_Count             0
Mobility_Rate         0
School_Rating         0
School_Grade          0
Owner_Occupied        0
Renter_Occupied       0
Total_Vacant          0
Total_Dwellings       0
FTE_Employed          0
Unemployed            0
Expense_Index         0
Average_Commute       0
Crime_Index           0
Loan_Amount           0
Loan_Term             0
Loan_R                0
Loan_Payment          0
Home_Affordability    0
dtype: int64